# Практическое задание 4
## Вариант 7
## Задание 1 

In [1]:
import pandas as pd
import numpy as np

data = ['ABCD','ABE','BC','ABD','BDE','AD','ADE','ABCDE']

In [2]:
def printSet(itemSets,tail=50):
    if itemSets != {}:
        df=pd.DataFrame()
        df['itemset']=[list(item) for item in itemSets.keys()]
        df['support']=itemSets.values()
        df['length'] = df['itemset'].apply(lambda x: len(x))
        print(df.tail(tail))

from collections import defaultdict

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    combItem = {}
    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                combItem.setdefault(item, 0)
                combItem[item] += 1
    num_items=10
    combItem = {item:val for item,val in combItem.items() if val/num_items >= minSupport}
    return combItem


def joinSet(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(dataset):
    transactionList = list( frozenset(transaction) for transaction in dataset)
    itemSet = set(frozenset([item]) for transaction in dataset for item in transaction)
    return itemSet, transactionList


def runApriori(data_iter, minSupport):
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    freqSet = defaultdict(int)
    resultSet = {}
    currentLSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)
    uniqSet=currentLSet
    resultSet.update(currentLSet)
    currentLSet = set(currentLSet.keys())
    k = 2
    while currentLSet != set([]): 
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport, freqSet)
        resultSet.update(currentCSet)
        currentLSet = set(currentCSet)
        k = k + 1
    return resultSet,uniqSet

In [3]:
print('Source data D =',data)
print('Min sup value',0.2)
result,uniqSet = runApriori(data,0.2)
print('\n1.1\nApriori result table:')
printSet(result)

def minGen(itemset,generset):
    for item,support in itemset.items():
        check=True
        for g_item,g_support in generset.items():
            if g_item.issubset(item):
                if support>=g_support:
                    check=False
        if check is not False: 
            generset.setdefault(item,support)
    return generset
print('\n1.2\nMin gen result table:')
printSet(minGen(result,uniqSet))

print('\n1.3\nAdd to D BCE')
data.append('E')
result1,uniqSet1=runApriori(data,0.2)
print('Apriori result table:')
printSet(result1)
print('\nMin gen result table:')
printSet(minGen(result1,uniqSet1))


Source data D = ['ABCD', 'ABE', 'BC', 'ABD', 'BDE', 'AD', 'ADE', 'ABCDE']
Min sup value 0.2

1.1
Apriori result table:
         itemset  support  length
0            [E]        4       1
1            [C]        3       1
2            [A]        6       1
3            [B]        6       1
4            [D]        6       1
5         [A, D]        5       2
6         [D, E]        3       2
7         [C, D]        2       2
8         [B, C]        3       2
9         [C, A]        2       2
10        [B, A]        4       2
11        [B, E]        3       2
12        [B, D]        4       2
13        [A, E]        3       2
14     [B, A, D]        3       3
15     [B, A, E]        2       3
16     [D, B, E]        2       3
17     [D, A, E]        2       3
18     [B, C, D]        2       3
19     [C, A, D]        2       3
20     [B, C, A]        2       3
21  [A, D, B, C]        2       4

1.2
Min gen result table:
      itemset  support  length
0         [E]        4       1
1         

## Задание 2

In [4]:
data_S = [('A',6),('B',7),('C',6),('D',8),('E',7),('AB',4),('AC',4),('AD',4),('AE',5),('BC',4),('BD',5),('BE',4),('CD',5),('CE',4),('DE',5),('ABC',3),('ABD',2)
         ,('ABE',3),('ACD',3),('ACE',3),('ADE',3),('BCD',3),('BCE',2),('BDE',2),('CDE',3),('ABCD',2),('ABCE',2),('ABDE',1),('ACDE',2),('BCDE',1),('ABCDE',1)]
print('Source data S =',data_S)
print('Min sup value',0.2)

Source data S = [('A', 6), ('B', 7), ('C', 6), ('D', 8), ('E', 7), ('AB', 4), ('AC', 4), ('AD', 4), ('AE', 5), ('BC', 4), ('BD', 5), ('BE', 4), ('CD', 5), ('CE', 4), ('DE', 5), ('ABC', 3), ('ABD', 2), ('ABE', 3), ('ACD', 3), ('ACE', 3), ('ADE', 3), ('BCD', 3), ('BCE', 2), ('BDE', 2), ('CDE', 3), ('ABCD', 2), ('ABCE', 2), ('ABDE', 1), ('ACDE', 2), ('BCDE', 1), ('ABCDE', 1)]
Min sup value 0.2


In [5]:
def recovT(data_S):
    data = dict(data_S)
    T=[]
    for r_item, r_sup in reversed(data.items()):
        if len(r_item) !=1:
            for item, sup in data.items():
                if set(item).issubset(set(r_item)) and len(item)<len(r_item) and r_sup!=0:
                    data[item]=sup-r_sup    
    for item,sup in data.items():
        if sup>0: T.append(item)
    return T

In [6]:
T = recovT(data_S)
print('2.1\nT=',T)

2.1
T= ['BD', 'ABE', 'ADE', 'BCD', 'BDE', 'CDE', 'ABCD', 'ABCE', 'ACDE', 'ABCDE']


In [7]:
def findIndex(data,T):
    temp_dict = dict(data)
    global_arr=[]
    for key in temp_dict:
        arr_ind=[]
        for index,item in enumerate(T):
            if set(key).issubset(set(item)):
                arr_ind.append(index)
        global_arr.append((key,arr_ind))
    return global_arr

In [8]:
P = findIndex(data_S,T)
print('2.2\nP =',P)

2.2
P = [('A', [1, 2, 6, 7, 8, 9]), ('B', [0, 1, 3, 4, 6, 7, 9]), ('C', [3, 5, 6, 7, 8, 9]), ('D', [0, 2, 3, 4, 5, 6, 8, 9]), ('E', [1, 2, 4, 5, 7, 8, 9]), ('AB', [1, 6, 7, 9]), ('AC', [6, 7, 8, 9]), ('AD', [2, 6, 8, 9]), ('AE', [1, 2, 7, 8, 9]), ('BC', [3, 6, 7, 9]), ('BD', [0, 3, 4, 6, 9]), ('BE', [1, 4, 7, 9]), ('CD', [3, 5, 6, 8, 9]), ('CE', [5, 7, 8, 9]), ('DE', [2, 4, 5, 8, 9]), ('ABC', [6, 7, 9]), ('ABD', [6, 9]), ('ABE', [1, 7, 9]), ('ACD', [6, 8, 9]), ('ACE', [7, 8, 9]), ('ADE', [2, 8, 9]), ('BCD', [3, 6, 9]), ('BCE', [7, 9]), ('BDE', [4, 9]), ('CDE', [5, 8, 9]), ('ABCD', [6, 9]), ('ABCE', [7, 9]), ('ABDE', [9]), ('ACDE', [8, 9]), ('BCDE', [9]), ('ABCDE', [9])]


In [9]:
print('2.3\n Start Charm with minsup value: 0.2')
def prepareP(P):
    P = dict(P)
    P_comb={}
    for item in P.copy():
        if len(item)>1:
            P_comb[item] = P[item]
            P.pop(item)
    return P,P_comb

def runCharm(P,P_comb,min_sup, C):
    print('\nStart runCharm')
    for item, value in P.items():
        p = {}
        for item_j, value_j in P.items():
            if list(P.keys()).index(item_j)>list(P.keys()).index(item):
                new_item=item+item_j
                if(P_comb.get(new_item)) is not None:
                    new_sup = len(P_comb[new_item])
                    if new_sup >= min_sup*10:
                        print(f'Current element: ({new_item},{P_comb[new_item]})')
                        if value == value_j:
                            P[new_item]=P.pop(item)
                            if item in p: p[new_item]=p.pop(item)
                            P.pop(item_j)
                        elif set(value).issubset(set(value_j)):
                            P[new_item]=P.pop(item)
                            if item in p: p[new_item]=p.pop(item)
                        else:
                            print(f'Add to Pi: ({new_item},{P_comb[new_item]})')
                            p.setdefault(new_item,P_comb[new_item])
        if len(p) != 0: runCharm(p,P_comb,min_sup,C)
        if len(C) != 0:
            for closed_item in C.copy():
                if set(item).issubset(set(closed_item)) is False:
                    C.setdefault(item,value)
                elif value != C[closed_item]:
                    C.setdefault(item,value)
        else: 
            C.setdefault(item,value)
        print(f'Add to C: ({item},{value})')
    C = sorted(C, key=len)
C={}
P,P_comb=prepareP(P)
runCharm(P,P_comb,0.2,C)
C = sorted(C, key=len)
print('\nC=',C)

2.3
 Start Charm with minsup value: 0.2

Start runCharm
Current element: (AB,[1, 6, 7, 9])
Add to Pi: (AB,[1, 6, 7, 9])
Current element: (AC,[6, 7, 8, 9])
Add to Pi: (AC,[6, 7, 8, 9])
Current element: (AD,[2, 6, 8, 9])
Add to Pi: (AD,[2, 6, 8, 9])
Current element: (AE,[1, 2, 7, 8, 9])
Add to Pi: (AE,[1, 2, 7, 8, 9])

Start runCharm
Add to C: (AB,[1, 6, 7, 9])
Add to C: (AC,[6, 7, 8, 9])
Add to C: (AD,[2, 6, 8, 9])
Add to C: (AE,[1, 2, 7, 8, 9])
Add to C: (A,[1, 2, 6, 7, 8, 9])
Current element: (BC,[3, 6, 7, 9])
Add to Pi: (BC,[3, 6, 7, 9])
Current element: (BD,[0, 3, 4, 6, 9])
Add to Pi: (BD,[0, 3, 4, 6, 9])
Current element: (BE,[1, 4, 7, 9])
Add to Pi: (BE,[1, 4, 7, 9])

Start runCharm
Add to C: (BC,[3, 6, 7, 9])
Add to C: (BD,[0, 3, 4, 6, 9])
Add to C: (BE,[1, 4, 7, 9])
Add to C: (B,[0, 1, 3, 4, 6, 7, 9])
Current element: (CD,[3, 5, 6, 8, 9])
Add to Pi: (CD,[3, 5, 6, 8, 9])
Current element: (CE,[5, 7, 8, 9])
Add to Pi: (CE,[5, 7, 8, 9])

Start runCharm
Add to C: (CD,[3, 5, 6, 8, 9])


In [10]:
print('2.4\n Add +1 to BC')
data=dict(data_S)
data['BC']+=1
print('N_Source data S',data)
T = recovT(data)
print('T=',T)
P = findIndex(data_S,T)
print('P=',P)
C={}
P,P_comb=prepareP(P)
runCharm(P,P_comb,0.2,C)
C = sorted(C, key=len)
print('\nC=',C)

2.4
 Add +1 to BC
N_Source data S {'A': 6, 'B': 7, 'C': 6, 'D': 8, 'E': 7, 'AB': 4, 'AC': 4, 'AD': 4, 'AE': 5, 'BC': 5, 'BD': 5, 'BE': 4, 'CD': 5, 'CE': 4, 'DE': 5, 'ABC': 3, 'ABD': 2, 'ABE': 3, 'ACD': 3, 'ACE': 3, 'ADE': 3, 'BCD': 3, 'BCE': 2, 'BDE': 2, 'CDE': 3, 'ABCD': 2, 'ABCE': 2, 'ABDE': 1, 'ACDE': 2, 'BCDE': 1, 'ABCDE': 1}
T= ['BC', 'BD', 'ABE', 'ADE', 'BCD', 'BDE', 'CDE', 'ABCD', 'ABCE', 'ACDE', 'ABCDE']
P= [('A', [2, 3, 7, 8, 9, 10]), ('B', [0, 1, 2, 4, 5, 7, 8, 10]), ('C', [0, 4, 6, 7, 8, 9, 10]), ('D', [1, 3, 4, 5, 6, 7, 9, 10]), ('E', [2, 3, 5, 6, 8, 9, 10]), ('AB', [2, 7, 8, 10]), ('AC', [7, 8, 9, 10]), ('AD', [3, 7, 9, 10]), ('AE', [2, 3, 8, 9, 10]), ('BC', [0, 4, 7, 8, 10]), ('BD', [1, 4, 5, 7, 10]), ('BE', [2, 5, 8, 10]), ('CD', [4, 6, 7, 9, 10]), ('CE', [6, 8, 9, 10]), ('DE', [3, 5, 6, 9, 10]), ('ABC', [7, 8, 10]), ('ABD', [7, 10]), ('ABE', [2, 8, 10]), ('ACD', [7, 9, 10]), ('ACE', [8, 9, 10]), ('ADE', [3, 9, 10]), ('BCD', [4, 7, 10]), ('BCE', [8, 10]), ('BDE', [5, 10]

## Задание 3

In [11]:
data=['TATTTATGGTATG','GTACTCTTAGTA','CCCGAGGTGAGCT','GTGTCACATATA','TCGTCTAAAAT']

In [12]:
def makeComb(itemSet,uniq_el):
    comb_item=[]
    for item in itemSet:
        for element in uniq_el:
            comb_item.append(item+element)
    return comb_item

def returnItemsWithMinSupport(itemSet, transactionList, minSupport):
    support_item={}
    for item in itemSet:
        for transaction in transactionList:
            proc_symb=0
            symb_item=0
            symb_trans=0
            while symb_item<len(item) and symb_trans<len(transaction):
                if item[symb_item]==transaction[symb_trans]:
                    proc_symb+=1         
                    symb_item+=1
                    symb_trans+=1
                else:
                    symb_trans+=1
            if proc_symb == len(item):
                support_item.setdefault(item,0)
                support_item[item]+=1
    sup_out=support_item.copy()
    for item in sup_out.copy():
        new_item=''.join(item)
        sup_out[new_item]=sup_out.pop(item)
    print('After comp sup: ',sup_out)
    support_item={item:val for item,val in support_item.items() if val/10 >= minSupport}
    print('Add to F:',support_item)
    return support_item

def getItemSetTransactionList(dataset):
    transactionList = []
    for transaction in dataset:
        temp_ar=[]
        for item in transaction:
            temp_ar.append(item)
        transactionList.append(temp_ar)           
    itemSet = set(item for transaction in dataset for item in transaction)
    return itemSet, transactionList


def runGSP(data_iter, minSupport):
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    print('Unique elements:',[list(item) for item in itemSet])
    resultSet = {}
    print('\nStep 1')
    currentLSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport)
    resultSet.update(currentLSet)
    currentLSet = list(currentLSet)
    k = 2
    while currentLSet != []: 
        items = list(set(item for transaction in currentLSet for item in transaction))
        currentLSet = makeComb(currentLSet,items)
        print('\nStep %d'% k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport)
        resultSet.update(currentCSet)
        currentLSet = list(currentCSet)
        k = k + 1
    return resultSet

In [13]:
print('Source data D =',data)
print('Min sup value',0.4)
print('3.1')
result = runGSP(data,0.4)
print('\nGSP result table:')
printSet(result,15)

Source data D = ['TATTTATGGTATG', 'GTACTCTTAGTA', 'CCCGAGGTGAGCT', 'GTGTCACATATA', 'TCGTCTAAAAT']
Min sup value 0.4
3.1
Unique elements: [['T'], ['C'], ['A'], ['G']]

Step 1
After comp sup:  {'T': 5, 'C': 4, 'A': 5, 'G': 5}
Add to F: {'T': 5, 'C': 4, 'A': 5, 'G': 5}

Step 2
After comp sup:  {'TT': 5, 'TC': 4, 'TA': 5, 'TG': 5, 'CT': 4, 'CC': 4, 'CA': 4, 'CG': 3, 'AT': 5, 'AC': 3, 'AA': 5, 'AG': 3, 'GT': 5, 'GC': 4, 'GA': 5, 'GG': 4}
Add to F: {'TT': 5, 'TC': 4, 'TA': 5, 'TG': 5, 'CT': 4, 'CC': 4, 'CA': 4, 'AT': 5, 'AA': 5, 'GT': 5, 'GC': 4, 'GA': 5, 'GG': 4}

Step 3
After comp sup:  {'TTT': 4, 'TTC': 3, 'TTA': 4, 'TTG': 2, 'TCT': 4, 'TCC': 3, 'TCA': 3, 'TCG': 2, 'TAT': 5, 'TAC': 3, 'TAA': 4, 'TAG': 3, 'TGT': 5, 'TGC': 3, 'TGA': 5, 'TGG': 2, 'CTT': 4, 'CTC': 3, 'CTA': 4, 'CTG': 2, 'CCT': 4, 'CCC': 1, 'CCA': 4, 'CCG': 2, 'CAT': 4, 'CAC': 2, 'CAA': 4, 'CAG': 2, 'ATT': 4, 'ATC': 2, 'ATA': 4, 'ATG': 3, 'AAT': 5, 'AAC': 1, 'AAA': 4, 'AAG': 3, 'GTT': 5, 'GTC': 4, 'GTA': 5, 'GTG': 4, 'GCT': 4,

In [14]:
print('3.2\nAdd to D TGAGAGGGA')
data.append('TGAGAGGGA')
result = runGSP(data,0.4)
print('\nGSP result table:')
printSet(result,15)

3.2
Add to D TGAGAGGGA
Unique elements: [['T'], ['C'], ['A'], ['G']]

Step 1
After comp sup:  {'T': 6, 'C': 4, 'A': 6, 'G': 6}
Add to F: {'T': 6, 'C': 4, 'A': 6, 'G': 6}

Step 2
After comp sup:  {'TT': 5, 'TC': 4, 'TA': 6, 'TG': 6, 'CT': 4, 'CC': 4, 'CA': 4, 'CG': 3, 'AT': 5, 'AC': 3, 'AA': 6, 'AG': 4, 'GT': 5, 'GC': 4, 'GA': 6, 'GG': 5}
Add to F: {'TT': 5, 'TC': 4, 'TA': 6, 'TG': 6, 'CT': 4, 'CC': 4, 'CA': 4, 'AT': 5, 'AA': 6, 'AG': 4, 'GT': 5, 'GC': 4, 'GA': 6, 'GG': 5}

Step 3
After comp sup:  {'TTT': 4, 'TTC': 3, 'TTA': 4, 'TTG': 2, 'TCT': 4, 'TCC': 3, 'TCA': 3, 'TCG': 2, 'TAT': 5, 'TAC': 3, 'TAA': 5, 'TAG': 4, 'TGT': 5, 'TGC': 3, 'TGA': 6, 'TGG': 3, 'CTT': 4, 'CTC': 3, 'CTA': 4, 'CTG': 2, 'CCT': 4, 'CCC': 1, 'CCA': 4, 'CCG': 2, 'CAT': 4, 'CAC': 2, 'CAA': 4, 'CAG': 2, 'ATT': 4, 'ATC': 2, 'ATA': 4, 'ATG': 3, 'AAT': 5, 'AAC': 1, 'AAA': 5, 'AAG': 4, 'AGT': 3, 'AGC': 1, 'AGA': 4, 'AGG': 3, 'GTT': 5, 'GTC': 4, 'GTA': 5, 'GTG': 4, 'GCT': 4, 'GCC': 2, 'GCA': 3, 'GCG': 1, 'GAT': 5, 'GAC': 